## Proms/Constraints testing

In [1]:
import sys; sys.path.insert(0,'/Users/ryan/github/prosodic/')
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
from cadence.imports import *

In [4]:
# !rm -rf {PATH_DB}
# pd.DataFrame(get_syllable_ld("whimsical"))

In [35]:
# doc=Para("and spends his prodigal wits in bootless rhyme")
# doc.sylls()

In [49]:
def w_stressed(df, is_s):
    return (1-is_s) * df.prom_stress.values
def w_stressed_l(df, is_s):
    return (1-is_s) * df.prom_lstress.values
def w_stressed_p(df, is_s):
    return (1-is_s) * df.prom_pstress.values
def w_stressed_t(df, is_s):
    return (1-is_s) * df.prom_tstress.values

def s_unstressed(df, is_s):
    return is_s * (1-df.prom_stress.values)
def s_unstressed_l(df, is_s):
    return is_s * (1-df.prom_lstress.values)
def s_unstressed_p(df, is_s):
    return is_s * (1-df.prom_pstress.values)
def s_unstressed_t(df, is_s):
    return is_s * (1-df.prom_tstress.values)


def w_peak(df, is_s):
    return (1-is_s) * df.prom_strength.values
def w_peak_p(df, is_s):
    return (1-is_s) * df.prom_pstrength.values
def s_trough(df, is_s):
    return is_s * (1-df.prom_strength.values)
def s_trough_p(df, is_s):
    return is_s * (1-df.prom_pstrength.values)


def is_disyllab_pos(is_s):
    return np.array([
        float(is_s[i] and ((i and is_s[i-1]) or (i+1<len(is_s) and is_s[i+1])))
        for i in range(len(is_s))
    ])

def di_unresolved_across(df,is_s):
    o=[np.nan] # need to start with non-viol since this applies only to second position of disyllab
    for i in range(1,len(df)):
        # for every 2 consecutive sylls...
        s1=is_s[i-1]
        s2=is_s[i]
        if s1!=s2:
            # not disyllabic
            o.append(np.nan)
        else:
            row1,row2=df.iloc[i-1],df.iloc[i]
            if (row1.sent_i, row1.word_i) == (row2.sent_i, row2.word_i):
                # disyllabic position is within not across words
                o.append(np.nan)
            else:
                if s1 and s2:
                    # disyllabic strong position immediately violates
                    o.append(1.0)
                else:
                    if not row1.word_isfunc or not row2.word_isfunc:
                        o.append(1.0)
                    else:
                        o.append(0.0)
    return o

def di_unresolved_within(df,is_s):
    o=[np.nan] # need to start with non-viol since this applies only to second position of disyllab
    for i in range(1,len(df)):
        # for every 2 consecutive sylls...
        s1=is_s[i-1]
        s2=is_s[i]
        if s1!=s2:
            # not disyllabic
            o.append(np.nan)
        else:
            row1,row2=df.iloc[i-1],df.iloc[i]
            if (row1.sent_i, row1.word_i) != (row2.sent_i, row2.word_i):
                # disyllabic position is across not wihtin words
                o.append(np.nan)
            else:
                # disyllabic position within word
                # first position mist be light and stressed
                if not (row1.prom_weight==0 and row1.prom_stress>0):
                    o.append(1.0)
                else:
                    o.append(0.0)
    return o



def bad_window(s,badwindow=(1,1)):
    wlen=len(badwindow)
    l=[]
    window=[]
    for x in s:
        window.append(x)
        l.append(int(tuple(window)==badwindow))
        if len(window)>=wlen:window.pop(0)
    return l

def clash(df, is_s):
    is_stressed=[int(x>0) for x in df.prom_stress]
    return no_window(is_stressed,badwindow=(1,1))
def lapse(df, is_s):
    is_stressed=[int(x>0) for x in df.prom_stress]
    return bad_window(is_stressed,badwindow=(0,0,0))


In [50]:
constraints=[
    w_stressed,
    w_stressed_l,
    w_stressed_p,
    w_stressed_t,
    
    s_unstressed,
    s_unstressed_l,
    s_unstressed_p,
    s_unstressed_t,

    w_peak,
    w_peak_p,

    s_trough,
    s_trough_p,
    
    di_unresolved_across,
    di_unresolved_within,

    clash,
    lapse
]

In [64]:
def parse_proms(df, mtr):
    mtr=mtr.replace('|','')
    df=df.iloc[:len(mtr)]
    is_s=np.array([float(x=='s') for x in mtr])
    # odf=pd.DataFrame()#.assign(slot_i=df.slot_i, meter=list(mtr))
    odf=pd.DataFrame(index=df.index)
    for cnstr in constraints:
        odf['*' + cnstr.__name__] = cnstr(df, is_s)
    return odf

In [65]:
doc1=Para("and spends his prodigal wits in bootless rhyme")
doc2=Para("and spends his whimsical wits in bootless rhyme")
# doc.sylls()

In [66]:
dfcombo1 = next(doc1.iter_combos())
dfcombo2 = next(doc2.iter_combos())
dfcombo1

,sent_i,sentpart_i,unit_i,line_i,word_i,word_pref,word_str,word_tok,word_ipa_i,word_ipa,syll_i,syll_str,syll_ipa,syll_stress,syll_weight,index,pos_prontype,mtree_ishead,prom_lstress,pos_upos,pos_mood,prom_pstrength,prom_tstress,pos_gender,dep_type,pos_person,word_depth,pos_xpos,word_nsyll,dep_head,prom_strength,pos_number,word_isfunc,prom_pstress,pos_verbform,prom_stress,pos_poss,pos_tense,pos_degree,prom_weight,combo_i
slot_i,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,,and,and,1,ænd,1,and,ænd,U,H,0,,NaN,0.0,CCONJ,,NaN,0.333333,,cc,,3,CC,1,2,NaN,,1,0.0,,0.0,,,,1.0,1
2,1,1,1,1,2,,spends,spends,1,'spɛndz,1,spends,'spɛndz,P,H,1,,NaN,1.0,VERB,Ind,NaN,0.666667,,root,3,4,VBZ,1,0,NaN,Sing,0,0.0,Fin,1.0,,Pres,,1.0,1
3,1,1,1,1,3,,his,his,1,hɪz,1,his,hɪz,U,H,2,Prs,0.0,0.0,PRON,,NaN,0.000000,Masc,nmod:poss,3,5,PRP$,1,5,NaN,Sing,1,0.0,,0.0,Yes,,,1.0,1
4,1,1,1,1,4,,prodigal,prodigal,1,'prɑ.dɪ.gəl,1,pro,'prɑ,P,L,3,,0.0,1.0,ADJ,,0.0,0.333333,,amod,,5,JJ,3,5,1.0,,0,0.0,,1.0,,,Pos,0.0,1
5,1,1,1,1,4,,prodigal,prodigal,1,'prɑ.dɪ.gəl,2,di,dɪ,U,L,4,,0.0,1.0,ADJ,,0.0,0.333333,,amod,,5,JJ,3,5,0.0,,0,0.0,,0.0,,,Pos,0.0,1
6,1,1,1,1,4,,prodigal,prodigal,1,'prɑ.dɪ.gəl,3,gal,gəl,U,H,5,,0.0,1.0,ADJ,,0.0,0.333333,,amod,,5,JJ,3,5,NaN,,0,0.0,,0.0,,,Pos,1.0,1
7,1,1,1,1,5,,wits,wits,1,'wɪts,1,wits,'wɪts,P,H,6,,1.0,1.0,NOUN,,1.0,0.666667,,obj,,5,NNS,1,2,NaN,Plur,0,1.0,,1.0,,,,1.0,1
8,1,1,1,1,6,,in,in,1,ɪn,1,in,ɪn,U,H,7,,NaN,0.0,ADP,,0.0,0.333333,,case,,5,IN,1,8,0.0,,1,0.0,,0.0,,,,1.0,1
9,1,1,1,1,7,,bootless,bootless,1,'bʉːt.lʌs,1,boo,'bʉːt,P,H,9,,0.0,1.0,ADJ,,0.0,0.666667,,amod,,6,JJ,2,8,1.0,,0,0.0,,1.0,,,Pos,1.0,1


In [68]:
meter='wswswwswsws'
# meter='wswsswswsws'
dfparse=parse_proms(dfcombo2,meter).assign(meter=list(meter))
dfparse['*total']=dfparse.sum(axis=1)
dfparse.join(dfcombo2)[['meter','syll_str','prom_weight','prom_stress','*di_unresolved_within','*di_unresolved_across','sent_i','word_i']]

,meter,syll_str,prom_weight,prom_stress,*di_unresolved_within,*di_unresolved_across,sent_i,word_i
slot_i,,,,,,,,
1,w,and,1.0,0.0,NaN,NaN,1,1
2,s,spends,1.0,1.0,NaN,NaN,1,2
3,w,his,1.0,0.0,NaN,NaN,1,3
4,s,whim,1.0,1.0,NaN,NaN,1,4
5,w,si,0.0,0.0,NaN,NaN,1,4
6,w,cal,1.0,0.0,1.0,NaN,1,4
7,s,wits,1.0,1.0,NaN,NaN,1,5
8,w,in,1.0,0.0,NaN,NaN,1,6
9,s,boo,1.0,1.0,NaN,NaN,1,7


In [70]:
doc=Para('the the the the the the the the the the')
doc=Para('bad bad bad bad bad bad bad bad bad bad')
doc=Para('the the the bad bad the the the bad bad')
dfcombo=next(doc.iter_combos())
meter='ws' * 5
dfparse=parse_proms(dfcombo,meter).assign(meter=list(meter))
dfparse['*total']=dfparse.sum(axis=1)
dfparse.join(dfcombo)[['meter','syll_str','prom_weight','prom_stress','*di_unresolved_within','*di_unresolved_across','*clash','*lapse']]

,meter,syll_str,prom_weight,prom_stress,*di_unresolved_within,*di_unresolved_across,*clash,*lapse
slot_i,,,,,,,,
1,w,the,0.0,0.0,NaN,NaN,0,0
2,s,the,0.0,0.0,NaN,NaN,0,0
3,w,the,0.0,0.0,NaN,NaN,0,1
4,s,bad,1.0,1.0,NaN,NaN,0,0
5,w,bad,1.0,1.0,NaN,NaN,1,0
6,s,the,0.0,0.0,NaN,NaN,0,0
7,w,the,0.0,0.0,NaN,NaN,0,0
8,s,the,0.0,0.0,NaN,NaN,0,1
9,w,bad,1.0,1.0,NaN,NaN,0,0


In [7]:
# pos_types=[['w'],['s'],['ww'],['ss']]
# pos_types=['w','s']#,'ww','ss']

def getlenparse(l): return sum(len(x) for x in l)

def iter_mpos(nsyll, starter=[], pos_types=None, maxS=METER_MAX_S, maxW=METER_MAX_W):
    if pos_types is None:
        wtypes = ['w'*n for n in range(1,maxW+1)]
        stypes = ['s'*n for n in range(1,maxS+1)]
        pos_types=[[x] for x in wtypes + stypes]
        
    news=[]
    for pos_type in pos_types:
        if starter and starter[-1][-1]==pos_type[0][0]: continue
        new = starter + pos_type
        # if starter: print(starter[-1][-1], pos_type[0][0], new)
        #if not is_ok_parse(new): continue
        if getlenparse(new)<=nsyll:
            news.append(new)
    
    # news = battle_parses(news)
    if news: yield from news
    # print('\n'.join('|'.join(x) for x in news))
    for new in news: yield from iter_mpos(nsyll, starter=new, pos_types=pos_types)


In [12]:
POSSPARSED={}
def get_poss_parses_cleverly(n,maxS=METER_MAX_S,maxW=METER_MAX_W):
    global POSSPARSED

    key=(n,maxS,maxW)
    if not key in POSSPARSED:    
        dfpars = pd.DataFrame([
            dict(
                npos=len(x),
                nsyll=getlenparse(x),
                meter='|'.join(x) + '|',
            )
            for x in iter_mpos(n)
            if getlenparse(x)<=n
        ])
        POSSPARSED[key]=dfpars
    return POSSPARSED[key]

def parse_cleverly(dfcombo,min_nsyll=4):
    bounded={}
    dfpars=get_poss_parses_cleverly(len(dfcombo))
    scored={}
    for nsyll,nsylldf in sorted(dfpars.groupby('nsyll')):
        if nsyll<min_nsyll: continue
        scored={}
        for meter in nsylldf.meter:
            exclude=False
            for bmeter in bounded:
                if meter.startswith(bmeter):
                    exclude=True
                    break
            if exclude: continue
            scored[meter] = parse_proms(dfcombo, meter)
        for mtr1 in scored:
            for mtr2 in scored:
                if mtr1>=mtr2: continue
                s1=scored[mtr1].sum()
                s2=scored[mtr2].sum()
                s1_ever_better_than_s2 = any(s1<s2)
                s2_ever_better_than_s1 = any(s2<s1)
                if s1_ever_better_than_s2 and not s2_ever_better_than_s1:
                    # s1 bounds s2
                    bounded[mtr2]=mtr1
                elif s2_ever_better_than_s1 and not s1_ever_better_than_s2:
                    # s2 bounds s1
                    bounded[mtr1]=mtr2
    
    odf=pd.concat(
        mdf.assign(parse=mtr[:-1])
        for mtr,mdf in scored.items()
    )
    return odf

In [13]:
def parse_units(self,**kwargs):
    units=list(self.iter_units(**kwargs))
    return pmap_iter(
        parse_unit_combos,
        units,
        num_proc=1
    )

def parse_unit_combos(
        dfunit,
        # keep_cols=['unit_i','combo_i','word_i','word_ipa_i','syll_i','slot_i'],
        index=False,
        **kwargs):
    o=[]
    for dfcombo in iter_combos(dfunit):
        ok_parses = parse_cleverly(dfcombo,**kwargs)
        combo_i = dfcombo.combo_i.iloc[0]
        o.append(ok_parses.assign(combo_i=combo_i))
    unit_i=dfunit.unit_i.iloc[0]
    odf=pd.concat(o).assign(unit_i=unit_i) if len(o) else pd.DataFrame()
    odf=setindex(odf) if index and len(odf) else odf
    return odf

In [14]:
# doc=Para(sonnet)
# next(parse_units(doc))
o=list(parse_units(doc))

Mapping parse_unit_combos() [x1]: 100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


In [15]:
odf=setindex(pd.concat(o))
odf

f_resolution  ...  no_w_stress_t
unit_i parse                combo_i slot_i                ...               
1      ss|w|ss|w|s|w|ss|w   1       1                NaN  ...       0.000000
                                    2                1.0  ...       0.000000
                                    3                NaN  ...       0.000000
                                    4                NaN  ...       0.000000
                                    5                NaN  ...       0.000000
...                                                  ...  ...            ...
       w|s|w|s|ww|s|w|s|w|s 2       7                NaN  ...       0.000000
                                    8                NaN  ...       0.333333
                                    9                NaN  ...       0.000000
                                    10               NaN  ...       0.666667
                                    11               NaN  ...       0.000000

[132 rows x 6 columns]

In [26]:
pd.Series([0,0,0,1]).mean()

0.25

In [22]:
pd.Series([0,0,0,1]).sum()

1

In [24]:
pd.Series([np.nan,np.nan,np.nan,1]).mean()

1.0

In [16]:
# odf.groupby(['unit_i','parse','combo_i']).sum()

In [17]:
rank_parses??

Signature: rank_parses(dfparses, **kwargs)
Docstring: <no docstring>
Source:   
def rank_parses(dfparses,**kwargs):
    odf=dfparses
    try:
        odf[TOTALCOL]=odf[[col for col in odf.columns if col.startswith('*') and col!=TOTALCOL]].sum(axis=1)
        odf['is_troch']=odf['parse'].apply(lambda x: int(x and x[0]=='s'))
        newrankdf=odf[['parse_i',TOTALCOL]].groupby('parse_i').sum().sort_values(TOTALCOL).reset_index()
        newrankdf['ranks_int']=pd.Series(list(range(len(newrankdf))))+1
        newrank=dict(zip(newrankdf.parse_i, newrankdf.ranks_int))
        odf['parse_rank']=odf.parse_i.apply(lambda x: int(newrank.get(x,0)))
        odf['num_parses']=odf['parse_rank'].max()
    except (KeyError,AttributeError) as e:
        pass
    return odf
File:      ~/github/cadence/cadence/parsers/metrics.py
Type:      function


In [36]:
# doc.units()